In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from utils import *

In [4]:
import tensorflow as tf
import keras

2024-01-13 10:15:11.049780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-13 10:15:11.049821: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
def create_conv_model(channels, length):
	input_shape = (length, channels)
	num_classes = 1

	input_layer = keras.layers.Input(input_shape)

	conv1 = keras.layers.Conv1D(filters=64, kernel_size=6, padding="same")(input_layer)
	conv1 = keras.layers.BatchNormalization()(conv1)
	conv1 = keras.layers.ReLU()(conv1)

	conv2 = keras.layers.Conv1D(filters=64, kernel_size=6, padding="same")(conv1)
	conv2 = keras.layers.BatchNormalization()(conv2)
	conv2 = keras.layers.ReLU()(conv2)

	conv3 = keras.layers.Conv1D(filters=64, kernel_size=6, padding="same")(conv2)
	conv3 = keras.layers.BatchNormalization()(conv3)
	conv3 = keras.layers.ReLU()(conv3)

	gap = keras.layers.GlobalAveragePooling1D()(conv3)

	output_layer = keras.layers.Dense(num_classes, activation="sigmoid")(gap)

	model = keras.models.Model(inputs=input_layer, outputs=output_layer)

	model.compile(
		optimizer="adam",
		loss="binary_crossentropy",
		metrics=["accuracy"],
	)

	return model

In [6]:
def create_simple_model(channels, length):
	num_classes = 1
 
	model = tf.keras.models.Sequential([
		tf.keras.layers.Dense(256, activation='relu', input_dim=channels*length),
		tf.keras.layers.Dense(128, activation='relu'),
		tf.keras.layers.Dense(64, activation='relu'),
		tf.keras.layers.Dense(num_classes, activation='sigmoid')
	])
	
	model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
		optimizer=tf.keras.optimizers.Adam(), # use Adam instead of SGD
		metrics=['accuracy']
	)

	return model

In [8]:
# train convolutional model

epochs = 60
batch_size = 32

# dataset specific
channels = 6

datasets = [300, 400, 500]

for data in datasets:
    # set datapoint length for reshaping
    length = data

    model = create_conv_model(channels, length)

    print("Training on dataset with ", data, "datapoints...")

    print("Loading training data...")
    (df, X_conv, y) = get_data_conv("train", data, channels)

    print("Training model...")
    model.fit(X_conv, y, 
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
        verbose=1
    )

    # get stats on test data
    print("Loading test data...")
    (df, X_test_conv, y_test) = get_data_conv("test", data, channels)

    print("Evaluating model...")
    result = model.evaluate(X_test_conv, y_test)
    print("Accuracy of model for " + str(data) + " datapoints: ", result[1])

    print("Saving model...")
    model.save("./models/conv_" + str(data) + ".keras")

Training on dataset with  300 datapoints...
Loading training data...
Reshaping training data...
Training model...
Epoch 1/60
775/775 [==============================] - 25s 32ms/step - loss: 0.1504 - accuracy: 0.9467 - val_loss: 0.7655 - val_accuracy: 0.7879
Epoch 2/60
775/775 [==============================] - 24s 31ms/step - loss: 0.0664 - accuracy: 0.9768 - val_loss: 0.6527 - val_accuracy: 0.8252
Epoch 3/60
775/775 [==============================] - 24s 31ms/step - loss: 0.0407 - accuracy: 0.9866 - val_loss: 0.9842 - val_accuracy: 0.7991
Epoch 4/60
775/775 [==============================] - 24s 31ms/step - loss: 0.0284 - accuracy: 0.9912 - val_loss: 1.4431 - val_accuracy: 0.7789
Epoch 5/60
775/775 [==============================] - 30s 39ms/step - loss: 0.0242 - accuracy: 0.9924 - val_loss: 1.3701 - val_accuracy: 0.8029
Epoch 6/60
775/775 [==============================] - 27s 35ms/step - loss: 0.0171 - accuracy: 0.9946 - val_loss: 1.6146 - val_accuracy: 0.7878
Epoch 7/60
775/775 [==

In [9]:
# train simple model

epochs = 60
batch_size = 32

# dataset specific
channels = 6

datasets = [50, 100, 200, 300, 400, 500]

for data in datasets:
    # set datapoint length for reshaping
    length = data

    model = create_simple_model(channels, length)

    print("Training on dataset with ", data, "datapoints...")

    print("Loading training data...")
    (df, X, y) = get_data("train", data)

    print("Training model...")
    model.fit(X, y, 
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
        verbose=1
    )

    # get stats on test data
    print("Loading test data...")
    (df, X_test, y_test) = get_data("test", data)

    print("Evaluating model...")
    result = model.evaluate(X_test, y_test)
    print("Accuracy of model for " + str(data) + " datapoints: ", result[1])

    print("Saving model...")
    model.save("./models/simple_" + str(data) + ".keras")

Training on dataset with  50 datapoints...
Loading training data...
Training model...
Epoch 1/60
789/789 [==============================] - 6s 2ms/step - loss: 0.3294 - accuracy: 0.8850 - val_loss: 0.6957 - val_accuracy: 0.7458
Epoch 2/60
789/789 [==============================] - 1s 2ms/step - loss: 0.2415 - accuracy: 0.9100 - val_loss: 0.7948 - val_accuracy: 0.7574
Epoch 3/60
789/789 [==============================] - 1s 2ms/step - loss: 0.2108 - accuracy: 0.9217 - val_loss: 0.7370 - val_accuracy: 0.7525
Epoch 4/60
789/789 [==============================] - 1s 2ms/step - loss: 0.1775 - accuracy: 0.9337 - val_loss: 0.9614 - val_accuracy: 0.7409
Epoch 5/60
789/789 [==============================] - 1s 2ms/step - loss: 0.1595 - accuracy: 0.9393 - val_loss: 1.0621 - val_accuracy: 0.7528
Epoch 6/60
789/789 [==============================] - 1s 2ms/step - loss: 0.1387 - accuracy: 0.9460 - val_loss: 1.1371 - val_accuracy: 0.7550
Epoch 7/60
789/789 [==============================] - 1s 2ms/s

In [ ]:
# 400 datapoints seems to provide the highest accuracy